In [ ]:
def make_prediction(model_dict, dates, ticker_symbols):
    '''
    
    '''
    pred_base_range = model_dict["pred_base_range"]
    
    # Check if the model was trained for all the provided ticker symbols:
    ticker_symbols = set(ticker_symbols)
    tickers_to_remove = []
    for ticker in ticker_symbols:
        if ticker not in model_dict["ticker_symbols"]:
            print(f"WARN: model {model_dict['name']} was not trained to predict prices for {ticker}!")
            tickers_to_remove.append(ticker)
    
    for ticker in tickers_to_remove:
            ticker_symbols.remove(ticker)
            
    # Check if the date ranges are outside the range which were used for training:
    dates = set(dates)
    dates_to_remove = []
    for date in dates:
        if (date > model_dict["start_date"]) and (date < model_dict["end_date"]):
            print(f"WARN: model {model_dict['name']}  was trained with the date {date} and therefore no predictions for that date can be made!")
            dates_to_remove.append(date)
    for date in dates_to_remove:
        dates.remove(date)
            
    # Initialize a result dictionary
    result = {}
    for ticker in ticker_symbols:
        result[ticker] = {}
        # sort dates so there is no need to predict a day twice
        for target_date in dates:
            print(f"Predicting {ticker} price for {target_date}") 
            start_date = datetime.today() - timedelta(days=pred_base_range)
            # if the date is to far in the future, multiple iterations are necessary
            
            delta = target_date - datetime.today() # as timedelta to predict

            # If no Adj close is available for today, today also needs to be predicted
            for i in range(1, delta.days + 2):
                day = datetime.today() + timedelta(days=i)
                print(day)
            
            
            available_data = get_stock_data(ticker_symbols = [ticker], start_date =  start_date, end_date = datetime.today(), date_index = True, columns="all")
            formated_data = tickers_as_columns(available_data)
            filtered_data = formated_data.filter([ticker])
            dataset = filtered_data.values
            scaler, y_test = scale_data(dataset)
            print(y_test.shape)
            
            prediction = 0
            result[ticker][target_date] = prediction
        
    return result